<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#모듈-설치" data-toc-modified-id="모듈-설치-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>모듈 설치</a></span></li><li><span><a href="#모델-생성" data-toc-modified-id="모델-생성-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>모델 생성</a></span></li><li><span><a href="#모델-저장" data-toc-modified-id="모델-저장-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>모델 저장</a></span></li><li><span><a href="#모델-불러오기" data-toc-modified-id="모델-불러오기-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>모델 불러오기</a></span></li><li><span><a href="#훈련-데이터-설정" data-toc-modified-id="훈련-데이터-설정-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>훈련 데이터 설정</a></span></li><li><span><a href="#전체-흐름" data-toc-modified-id="전체-흐름-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>전체 흐름</a></span></li><li><span><a href="#훈련-순서" data-toc-modified-id="훈련-순서-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>훈련 순서</a></span></li><li><span><a href="#이어서..." data-toc-modified-id="이어서...-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>이어서...</a></span></li></ul></div>

# keras를 사용하여 암호화화폐 예측하기 - 2. 모델 저장 및 불러오기

모델을 저장하고, 불러오기를 반복하며 학습을 진행합니다. 이전 코드를 바탕으로 새롭게 작성하여 진행합니다.

모델을 생성하기 이전에 저장한 모델이 있는지 확인해서, 있으면 저장된 모델을 불러옵니다.

## 모듈 설치

모델을 저장하기 위해 `h5py` 모듈을 설치합니다.

jupyter 셀에서 쉘커맨드를 실행할 경우 앞에 `!` 를 붙여서 실행합니다.

In [1]:
!pip3 install h5py

## 모델 생성

모델을 생성합니다. 이전 코드와 동일합니다.

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

batch_size = 1
look_back = 10
feature = 2

model = Sequential()

for i in range(3):
    model.add(LSTM(32, batch_input_shape=(batch_size, look_back, feature), stateful=True, return_sequences=True))
    model.add(Dropout(0.1))
model.add(LSTM(32, batch_input_shape=(batch_size, look_back, feature), stateful=True))
model.add(Dropout(0.1))
model.add(Dense(feature))

model.compile(loss='mean_squared_error', optimizer='adam')

Using TensorFlow backend.


## 모델 저장

생성한 모델을 저장합니다. 일단, 파일명은 `xrp_hour.h5`로 지정하겠습니다.

In [3]:
model_name = "xrp_hour.h5"
model.save(model_name)

## 모델 불러오기

불러올 때는 `load_model`을 사용하면 됩니다.

In [4]:
from keras.models import load_model
model = load_model(model_name)

## 훈련 데이터 설정

훈련에 사용할 데이터를 불러오겠습니다.

In [5]:
import pandas as pd

file_name = 'hour_Bithumb_XRP_KRW.csv'
data = pd.read_csv(file_name)

high_low_data = data[['high','low']]
data = high_low_data.dropna(axis=0, how='any')

min_low = data.min(axis=0).low
data -= min_low
max_high = data.max(axis=0).high
data /= max_high

data_arr = data.values

훈련을 위한 입출력데이터를 생성합니다.

In [6]:
import numpy as np

data_x, data_y = [], []
for i in range(len(data_arr) - look_back):
    data_x.append(data_arr[i:(i+look_back)])
    data_y.append(data_arr[i+look_back])

x_train = np.array(data_x)
y_train = np.array(data_y)

훈련이 되는지 확인합니다.

In [7]:
model.fit(x_train, y_train, epochs=1, batch_size=1, shuffle=False)

Epoch 1/1
327/327 [==============================] - 31s 93ms/step - loss: 0.0196


## 전체 흐름

전체적인 흐름을 다시 정리합니다.
1. 저장된 모델이 있는지 확인
  - 있으면 불러오고
  - 없으면 생성
2. 훈련을 하고
3. 저장합니다.

이 과정을 반복하여 훈련을 지속할 수 있습니다.

모델을 생성하는 함수를 생성합니다. 마지막 부분에서 생성한 모델을 반환합니다.

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

def create_model():
    batch_size = 1
    look_back = 10
    feature = 2

    model = Sequential()
    depth = 3
    for i in range(depth):
        model.add(LSTM(((depth - i ) * 32), batch_input_shape=(batch_size, look_back, feature), stateful=True, return_sequences=True))
        model.add(Dropout(0.1))
    model.add(LSTM(32, batch_input_shape=(batch_size, look_back, feature), stateful=True))
    model.add(Dropout(0.1))
    model.add(Dense(feature))

    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

모델을 생성하는 함수를 만들고, 저장된 모델을 불러오는 함수를 생성합니다.

> 모듈을 import하는 부분은 위에서 실행하였으면, 다시 실행할 필요는 없습니다.

In [9]:
from keras.models import load_model

def load_saved_model():
    try:
        model = load_model(model_name)
    except:
        model = create_model()
    return model

저장된 모델을 불러보고 확인해 보겠습니다.

In [10]:
model = load_saved_model()
model

저장된 모델을 지워보고 확인해 보겠습니다. 파일이 있으면 지워지고, 없으면 `FileNotFoundError`가 발생할 것입니다.

In [11]:
import os, sys
try:
    os.remove(model_name)
except:
    print("Unexpected error:", sys.exc_info()[0])
model = load_saved_model()
model

## 훈련 순서

1. 모델 불러오기
2. 훈련
3. 모델 저장하기

In [12]:
model = load_saved_model()
model.fit(x_train, y_train, epochs=1, batch_size=1, shuffle=False)
model.save(model_name)

Epoch 1/1
327/327 [==============================] - 32s 96ms/step - loss: 0.0198


첫번째 값을 예측한 값과 비교해보겠습니다.

In [13]:
xhat = x_train[0]
prediction = model.predict(np.array([xhat]), batch_size=1)
print("y:", y_train[0])
print("p:", prediction)

y: [ 0.88607595  0.84599156]
p: [[ 0.18023619  0.2006087 ]]


동작을 확인하였으면, 훈련을 하는 함수를 생성합니다.

In [14]:
def train_model(train_count):
    model = load_saved_model()
    for i in range(train_count):
        print("train_step:{}".format(i+1))
        model.fit(x_train, y_train, epochs=1, batch_size=1, shuffle=False)
        model.reset_states()
    model.save(model_name)

훈련 함수를 호출합니다. 파라미터로 훈련 횟수를 설정합니다.

In [15]:
train_model(5)

train_step:1
Epoch 1/1
327/327 [==============================] - 26s 80ms/step - loss: 0.0158
train_step:2
Epoch 1/1
327/327 [==============================] - 25s 77ms/step - loss: 0.0196
train_step:3
Epoch 1/1
327/327 [==============================] - 26s 80ms/step - loss: 0.0250
train_step:4
Epoch 1/1
327/327 [==============================] - 26s 80ms/step - loss: 0.0340
train_step:5
Epoch 1/1
327/327 [==============================] - 26s 80ms/step - loss: 0.0329


## 이어서...

훈련된 모델을 사용하여 이후의 값을 예측해보겠습니다.